# Import Data

In [1]:
# clear memory
%reset -f

# helper.py
from helper import *

In [2]:
# load data
df = pd.read_csv('./data/clean/fantasy_data.csv')

# ensure values are sorted properly
df = df.sort_values(by=['Key', 'Exp']).reset_index(drop=True)

# drop non-normalized columns
df = df.drop(columns=['G', 'GS', 'ProBowl', 'AllPro', 'Pass_Cmp', 'Pass_Att', 'Pass_Yds', 'Pass_TD', 'Pass_Int', 'Rush_Att', 'Rush_Yds', 'Rush_Y/A', 'Rush_TD', 
                      'num_games', 'Touches', 'Rec_Tgt', 'Rec_Rec', 'Rec_Yds', 'Rec_Y/R', 'Rec_TD', 'Fmb', 'FmbLost', 'Scrim_TD', 'Pass_Y/A', 'Scrim_Yds', 
                      'Points_half-ppr', 'PointsOvrRank_half-ppr', 'PointsPosRank_half-ppr', 'Points_VORP_half-ppr', 'PointsTarget_half-ppr'])

# show shape and nulls
show_shape_and_nulls(df)

Shape: (29369, 36)
Null values:


,Player,Tm,Pos,Age,Key,Year,games_played_pct,games_started_pct,Exp,Pass_Cmp%,Rec_Catch%,Pass_Cmp_per_game,Pass_Att_per_game,Pass_Yds_per_game,Pass_TD_per_game,Pass_Int_per_game,Rush_Att_per_game,Rush_Yds_per_game,Rush_TD_per_game,Rec_Tgt_per_game,Rec_Rec_per_game,Rec_Yds_per_game,Rec_TD_per_game,Fmb_per_game,FmbLost_per_game,Scrim_TD_per_game,Scrim_Yds_per_game,Touches_per_game,PPG_half-ppr,PPT_half-ppr,PPGOvrRank_half-ppr,PPGPosRank_half-ppr,PPTOvrRank_half-ppr,PPTPosRank_half-ppr,PPG_VORP_half-ppr,PPGTarget_half-ppr
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6959


- We will be using normalized features (per-game stats and other percentages) along with __PPG half-ppr fantasy points__ as the target. 
- This will not punish players for missing games or getting injured (which is unfortunate but very common in the NFL).
- A [seperate model]() will be trained to determine injury probability based on past volume and games missed.
- The only null values are held in the "Target" column (indicating players in their final season). We cannot use rows without a target to train, so they will be dropped here.

In [ ]:
# drop players with null target values
df = df.dropna(subset='PPGTarget_half-ppr')

# check
df.isna().sum().sum()

0

# Create Features

In [4]:
# aggregate data by player
features = create_features(df)

# check
features.sample()

,Age,AllPro,AllPro_3y_mean,AllPro_3y_std,AllPro_6y_mean,AllPro_6y_std,AllPro_career_mean,AllPro_career_std,AllPro_career_trend_slope,AllPro_momentum,AllPro_pct_change_prev,Exp,Fmb,FmbLost,FmbLost_3y_mean,FmbLost_3y_std,FmbLost_6y_mean,FmbLost_6y_std,FmbLost_career_mean,FmbLost_career_std,FmbLost_career_trend_slope,FmbLost_momentum,FmbLost_pct_change_prev,FmbLost_per_game,FmbLost_per_game_3y_mean,FmbLost_per_game_3y_std,FmbLost_per_game_6y_mean,FmbLost_per_game_6y_std,FmbLost_per_game_career_mean,FmbLost_per_game_career_std,FmbLost_per_game_career_trend_slope,FmbLost_per_game_momentum,FmbLost_per_game_pct_change_prev,Fmb_3y_mean,Fmb_3y_std,Fmb_6y_mean,Fmb_6y_std,Fmb_career_mean,Fmb_career_std,Fmb_career_trend_slope,Fmb_momentum,Fmb_pct_change_prev,Fmb_per_game,Fmb_per_game_3y_mean,Fmb_per_game_3y_std,Fmb_per_game_6y_mean,Fmb_per_game_6y_std,Fmb_per_game_career_mean,Fmb_per_game_career_std,Fmb_per_game_career_trend_slope,Fmb_per_game_momentum,Fmb_per_game_pct_change_prev,G,GS,GS_3y_mean,GS_3y_std,GS_6y_mean,GS_6y_std,GS_career_mean,GS_career_std,GS_career_trend_slope,GS_momentum,GS_pct_change_prev,G_3y_mean,G_3y_std,G_6y_mean,G_6y_std,G_career_mean,G_career_std,G_career_trend_slope,G_momentum,G_pct_change_prev,Key,PPGOvrRank_half-ppr,PPGOvrRank_half-ppr_3y_mean,PPGOvrRank_half-ppr_3y_std,PPGOvrRank_half-ppr_6y_mean,PPGOvrRank_half-ppr_6y_std,PPGOvrRank_half-ppr_career_mean,PPGOvrRank_half-ppr_career_std,PPGOvrRank_half-ppr_career_trend_slope,PPGOvrRank_half-ppr_momentum,PPGOvrRank_half-ppr_pct_change_prev,PPGPosRank_half-ppr,PPGPosRank_half-ppr_3y_mean,PPGPosRank_half-ppr_3y_std,PPGPosRank_half-ppr_6y_mean,PPGPosRank_half-ppr_6y_std,PPGPosRank_half-ppr_career_mean,PPGPosRank_half-ppr_career_std,PPGPosRank_half-ppr_career_trend_slope,PPGPosRank_half-ppr_momentum,PPGPosRank_half-ppr_pct_change_prev,PPG_VORP_half-ppr,PPG_VORP_half-ppr_3y_mean,PPG_VORP_half-ppr_3y_std,PPG_VORP_half-ppr_6y_mean,PPG_VORP_half-ppr_6y_std,PPG_VORP_half-ppr_career_mean,PPG_VORP_half-ppr_career_std,PPG_VORP_half-ppr_career_trend_slope,PPG_VORP_half-ppr_momentum,PPG_VORP_half-ppr_pct_change_prev,PPG_half-ppr,PPG_half-ppr_3y_mean,PPG_half-ppr_3y_std,PPG_half-ppr_6y_mean,PPG_half-ppr_6y_std,PPG_half-ppr_career_mean,PPG_half-ppr_career_std,PPG_half-ppr_career_trend_slope,PPG_half-ppr_momentum,PPG_half-ppr_pct_change_prev,PPTOvrRank_half-ppr,PPTOvrRank_half-ppr_3y_mean,PPTOvrRank_half-ppr_3y_std,PPTOvrRank_half-ppr_6y_mean,PPTOvrRank_half-ppr_6y_std,PPTOvrRank_half-ppr_career_mean,PPTOvrRank_half-ppr_career_std,PPTOvrRank_half-ppr_career_trend_slope,PPTOvrRank_half-ppr_momentum,PPTOvrRank_half-ppr_pct_change_prev,PPTPosRank_half-ppr,PPTPosRank_half-ppr_3y_mean,PPTPosRank_half-ppr_3y_std,PPTPosRank_half-ppr_6y_mean,PPTPosRank_half-ppr_6y_std,PPTPosRank_half-ppr_career_mean,PPTPosRank_half-ppr_career_std,PPTPosRank_half-ppr_career_trend_slope,PPTPosRank_half-ppr_momentum,PPTPosRank_half-ppr_pct_change_prev,PPT_half-ppr,PPT_half-ppr_3y_mean,PPT_half-ppr_3y_std,PPT_half-ppr_6y_mean,PPT_half-ppr_6y_std,PPT_half-ppr_career_mean,PPT_half-ppr_career_std,PPT_half-ppr_career_trend_slope,PPT_half-ppr_momentum,PPT_half-ppr_pct_change_prev,Pass_Att,Pass_Att_3y_mean,Pass_Att_3y_std,Pass_Att_6y_mean,Pass_Att_6y_std,Pass_Att_career_mean,Pass_Att_career_std,Pass_Att_career_trend_slope,Pass_Att_momentum,Pass_Att_pct_change_prev,Pass_Att_per_game,Pass_Att_per_game_3y_mean,Pass_Att_per_game_3y_std,Pass_Att_per_game_6y_mean,Pass_Att_per_game_6y_std,Pass_Att_per_game_career_mean,Pass_Att_per_game_career_std,Pass_Att_per_game_career_trend_slope,Pass_Att_per_game_momentum,Pass_Att_per_game_pct_change_prev,Pass_Cmp,Pass_Cmp%,Pass_Cmp%_3y_mean,Pass_Cmp%_3y_std,Pass_Cmp%_6y_mean,Pass_Cmp%_6y_std,Pass_Cmp%_career_mean,Pass_Cmp%_career_std,Pass_Cmp%_career_trend_slope,Pass_Cmp%_momentum,Pass_Cmp%_pct_change_prev,Pass_Cmp_3y_mean,Pass_Cmp_3y_std,Pass_Cmp_6y_mean,Pass_Cmp_6y_std,Pass_Cmp_career_mean,Pass_Cmp_career_std,Pass_Cmp_career_trend_slope,Pass_Cmp_momentum,Pass_Cmp_pct_change_prev,P

# Positional Subsets

In [5]:
# turn 'Pos' column into 4 categorical columns
features = pd.get_dummies(features, columns=['Pos'])

# create the 4 positional subsets
qb = features.query('Pos_QB')
rb = features.query('Pos_RB')
wr_te = features.query('Pos_WR | Pos_TE')

# drop positional columns
pos_cols = ['Pos_QB', 'Pos_RB', 'Pos_WR', 'Pos_TE']
qb = qb.drop(columns=pos_cols)
rb = rb.drop(columns=pos_cols)
wr_te = wr_te.drop(columns=pos_cols)

# drop 'Rec' cols for QBs
rec_cols = [col for col in features.columns if col.startswith('Rec_')]
qb = qb.drop(columns=rec_cols)

# drop 'Pass' cols for RBs and WRs/TEs
pass_cols = [col for col in features.columns if col.startswith('Pass_')]
for pos in [rb, wr_te]:
    pos = pos.drop(columns=pass_cols)

# show shapes
qb.shape, rb.shape, wr_te.shape

((3458, 467), (7089, 567), (11863, 567))

# Baseline
- Baseline RMSE scores for the 3 positional subsets using an XGBoost with 1000 trees.

In [6]:
# iterate through positional subsets 3 + 6yr with percent change
for pos, data in zip(['QB', 'RB', 'WR/TE'], [qb, rb, wr_te]):
    # cross validate
    model = XGBRegressor(n_jobs=-1, random_state=SEED, n_estimators=1000)
    summary = cross_val(data, model)

    # show results
    print(f'--- {pos} ---')
    display(summary)

--- QB ---


,train_rmse,val_rmse,train_r2,val_r2
mean,0.000923,74.441792,1.000000e+00,0.440381
std,0.000063,0.992166,1.156454e-11,0.021269


--- RB ---


,train_rmse,val_rmse,train_r2,val_r2
mean,1.213199,57.475809,0.999672,0.401695
std,0.668283,2.019876,0.000261,0.032271


--- WR/TE ---


,train_rmse,val_rmse,train_r2,val_r2
mean,1.324868,45.479073,0.999546,0.475005
std,0.200307,0.910169,0.000129,0.020190


In [7]:
# iterate through positional subsets rookie change
for pos, data in zip(['QB', 'RB', 'WR/TE'], [qb, rb, wr_te]):
    # cross validate
    model = XGBRegressor(n_jobs=-1, random_state=SEED, n_estimators=1000)
    summary = cross_val(data, model)

    # show results
    print(f'--- {pos} ---')
    display(summary)

--- QB ---


,train_rmse,val_rmse,train_r2,val_r2
mean,0.00090,74.402372,1.000000e+00,0.440880
std,0.00003,0.657471,5.977447e-12,0.022827


--- RB ---


,train_rmse,val_rmse,train_r2,val_r2
mean,1.213207,57.836185,0.999672,0.394017
std,0.668275,1.600985,0.000261,0.030773


--- WR/TE ---


,train_rmse,val_rmse,train_r2,val_r2
mean,1.328009,45.645595,0.999544,0.471199
std,0.199373,0.940233,0.000129,0.019148


In [6]:
# Joe Burrow
qb.query('Player == "Joe Burrow"')

,Age,AllPro,AllPro_3y_mean,AllPro_3y_std,AllPro_6y_mean,AllPro_6y_std,AllPro_career_mean,AllPro_career_std,AllPro_career_trend_slope,AllPro_momentum,AllPro_pct_change_prev,Exp,Fmb,FmbLost,FmbLost_3y_mean,FmbLost_3y_std,FmbLost_6y_mean,FmbLost_6y_std,FmbLost_career_mean,FmbLost_career_std,FmbLost_career_trend_slope,FmbLost_momentum,FmbLost_pct_change_prev,FmbLost_per_game,FmbLost_per_game_3y_mean,FmbLost_per_game_3y_std,FmbLost_per_game_6y_mean,FmbLost_per_game_6y_std,FmbLost_per_game_career_mean,FmbLost_per_game_career_std,FmbLost_per_game_career_trend_slope,FmbLost_per_game_momentum,FmbLost_per_game_pct_change_prev,Fmb_3y_mean,Fmb_3y_std,Fmb_6y_mean,Fmb_6y_std,Fmb_career_mean,Fmb_career_std,Fmb_career_trend_slope,Fmb_momentum,Fmb_pct_change_prev,Fmb_per_game,Fmb_per_game_3y_mean,Fmb_per_game_3y_std,Fmb_per_game_6y_mean,Fmb_per_game_6y_std,Fmb_per_game_career_mean,Fmb_per_game_career_std,Fmb_per_game_career_trend_slope,Fmb_per_game_momentum,Fmb_per_game_pct_change_prev,G,GS,GS_3y_mean,GS_3y_std,GS_6y_mean,GS_6y_std,GS_career_mean,GS_career_std,GS_career_trend_slope,GS_momentum,GS_pct_change_prev,G_3y_mean,G_3y_std,G_6y_mean,G_6y_std,G_career_mean,G_career_std,G_career_trend_slope,G_momentum,G_pct_change_prev,Key,PPGOvrRank_half-ppr,PPGOvrRank_half-ppr_3y_mean,PPGOvrRank_half-ppr_3y_std,PPGOvrRank_half-ppr_6y_mean,PPGOvrRank_half-ppr_6y_std,PPGOvrRank_half-ppr_career_mean,PPGOvrRank_half-ppr_career_std,PPGOvrRank_half-ppr_career_trend_slope,PPGOvrRank_half-ppr_momentum,PPGOvrRank_half-ppr_pct_change_prev,PPGPosRank_half-ppr,PPGPosRank_half-ppr_3y_mean,PPGPosRank_half-ppr_3y_std,PPGPosRank_half-ppr_6y_mean,PPGPosRank_half-ppr_6y_std,PPGPosRank_half-ppr_career_mean,PPGPosRank_half-ppr_career_std,PPGPosRank_half-ppr_career_trend_slope,PPGPosRank_half-ppr_momentum,PPGPosRank_half-ppr_pct_change_prev,PPG_VORP_half-ppr,PPG_VORP_half-ppr_3y_mean,PPG_VORP_half-ppr_3y_std,PPG_VORP_half-ppr_6y_mean,PPG_VORP_half-ppr_6y_std,PPG_VORP_half-ppr_career_mean,PPG_VORP_half-ppr_career_std,PPG_VORP_half-ppr_career_trend_slope,PPG_VORP_half-ppr_momentum,PPG_VORP_half-ppr_pct_change_prev,PPG_half-ppr,PPG_half-ppr_3y_mean,PPG_half-ppr_3y_std,PPG_half-ppr_6y_mean,PPG_half-ppr_6y_std,PPG_half-ppr_career_mean,PPG_half-ppr_career_std,PPG_half-ppr_career_trend_slope,PPG_half-ppr_momentum,PPG_half-ppr_pct_change_prev,PPTOvrRank_half-ppr,PPTOvrRank_half-ppr_3y_mean,PPTOvrRank_half-ppr_3y_std,PPTOvrRank_half-ppr_6y_mean,PPTOvrRank_half-ppr_6y_std,PPTOvrRank_half-ppr_career_mean,PPTOvrRank_half-ppr_career_std,PPTOvrRank_half-ppr_career_trend_slope,PPTOvrRank_half-ppr_momentum,PPTOvrRank_half-ppr_pct_change_prev,PPTPosRank_half-ppr,PPTPosRank_half-ppr_3y_mean,PPTPosRank_half-ppr_3y_std,PPTPosRank_half-ppr_6y_mean,PPTPosRank_half-ppr_6y_std,PPTPosRank_half-ppr_career_mean,PPTPosRank_half-ppr_career_std,PPTPosRank_half-ppr_career_trend_slope,PPTPosRank_half-ppr_momentum,PPTPosRank_half-ppr_pct_change_prev,PPT_half-ppr,PPT_half-ppr_3y_mean,PPT_half-ppr_3y_std,PPT_half-ppr_6y_mean,PPT_half-ppr_6y_std,PPT_half-ppr_career_mean,PPT_half-ppr_career_std,PPT_half-ppr_career_trend_slope,PPT_half-ppr_momentum,PPT_half-ppr_pct_change_prev,Pass_Att,Pass_Att_3y_mean,Pass_Att_3y_std,Pass_Att_6y_mean,Pass_Att_6y_std,Pass_Att_career_mean,Pass_Att_career_std,Pass_Att_career_trend_slope,Pass_Att_momentum,Pass_Att_pct_change_prev,Pass_Att_per_game,Pass_Att_per_game_3y_mean,Pass_Att_per_game_3y_std,Pass_Att_per_game_6y_mean,Pass_Att_per_game_6y_std,Pass_Att_per_game_career_mean,Pass_Att_per_game_career_std,Pass_Att_per_game_career_trend_slope,Pass_Att_per_game_momentum,Pass_Att_per_game_pct_change_prev,Pass_Cmp,Pass_Cmp%,Pass_Cmp%_3y_mean,Pass_Cmp%_3y_std,Pass_Cmp%_6y_mean,Pass_Cmp%_6y_std,Pass_Cmp%_career_mean,Pass_Cmp%_career_std,Pass_Cmp%_career_trend_slope,Pass_Cmp%_momentum,Pass_Cmp%_pct_change_prev,Pass_Cmp_3y_mean,Pass_Cmp_3y_std,Pass_Cmp_6y_mean,Pass_Cmp_6y_std,Pass_Cmp_career_mean,Pass_Cmp_career_std,Pass_Cmp_career_trend_slope,Pass_Cmp_momentum,Pass_Cmp_pct_change_prev,P

- The train metrics are almost perfect, with much worse validation scores.
- We will run 100 iterations of bayesian optimization on each positional group to regularize and bring the validation scores down.

# Optimize XGBoost

In [ ]:
# non-feature cols
non_feat_cols = ['Player', 'Tm', 'Key', 'Year', 'num_games', 'PointsTarget_half-ppr']

# define X and y
X_qb = qb.drop(non_feat_cols, axis=1)
y_qb = qb['PointsTarget_half-ppr']
X_rb = rb.drop(non_feat_cols, axis=1)
y_rb = rb['PointsTarget_half-ppr']
X_wr_te = wr_te.drop(non_feat_cols, axis=1)
y_wr_te = wr_te['PointsTarget_half-ppr']

# define the parameter search space
param_bounds = {'max_depth': (1, 10),
    'learning_rate': (0.001, 0.1),
    'gamma': (0, 0.7),
    'subsample': (0.3, 1.0),
    'colsample_bytree': (0.3, 1.0),
    'min_child_weight': (1, 10)}

# bayesian optimize
optim_qb = run_xgb_bayesopt(X_qb, y_qb, param_bounds, SEED)
optim_rb = run_xgb_bayesopt(X_rb, y_rb, param_bounds, SEED)
optim_wr_te = run_xgb_bayesopt(X_wr_te, y_wr_te, param_bounds, SEED)

In [ ]:
optimizer_qb.max['params']

{'colsample_bytree': 0.9311038798928082,
 'gamma': 0.1659549803896018,
 'learning_rate': 0.04583336248385127,
 'max_depth': 4.11460555388863,
 'min_child_weight': 2.5128620790443588,
 'subsample': 0.5655932101749224}

In [ ]:
# get best params
best_params_qb = {'colsample_bytree': 0.9311038798928082,
 'gamma': 0.1659549803896018,
 'learning_rate': 0.04583336248385127,
 'max_depth': 4.11460555388863,
 'min_child_weight': 2,
 'subsample': 0.5655932101749224}
best_params_rb = {'colsample_bytree': 1.0,
 'gamma': 0.7,
 'learning_rate': 0.11222774354720268,
 'max_depth': 5.762287747574579,
 'min_child_weight': 6.966728084976303,
 'subsample': 1.0}
best_params_wr_te = 

# 2025 Predictions